# **ARMA and Linear Processes**

In [ ]:
import itertools, warnings, os, subprocess
import pandas as pd
import numpy as np
from random import gauss
import matplotlib.pyplot as plt
import warnings
from pandas.plotting import autocorrelation_plot
import os
from IPython.display import Image

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_process import ArmaProcess

from sklearn.metrics import mean_squared_error

from pmdarima.arima import auto_arima
from pmdarima import pipeline
from pmdarima import model_selection
from pmdarima import preprocessing as ppc
from pmdarima import arima

from prophet import Prophet 

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

warnings.simplefilter(action='ignore', category= FutureWarning)

In [ ]:
data = 'data/'

class CFG:
    img_dim1 = 12
    img_dim2 = 7
    fontsize = 9
    marker = 2
    lines = 2

# plt.rcParams.keys() to list params
# adjust the parameters for displayed figures    
plt.rcParams.update({'figure.figsize': (CFG.img_dim1,CFG.img_dim2),
                     'font.size': (CFG.fontsize),
                     'lines.markersize': (CFG.marker),
                     'lines.linewidth': (CFG.lines)})    

## **Basic Linear Processes**

**- ADD PROSE -**

### **Autoregressive Processes**

**-ADD PROSE-**

In [ ]:
?ArmaProcess

In [ ]:
ar1 = np.array([1, -0.9])
ma1 = np.array([1])

AR_object1 = ArmaProcess(ar1, ma1)
simulated_data1 = AR_object1.generate_sample(nsample=1000)
plt.plot(simulated_data1)
plt.show();

In [ ]:
# Plot ACF
plot_acf(simulated_data1, lags=10)
plt.show();

In [ ]:
# Plot PACF
plot_pacf(simulated_data1, lags=10)
plt.show();

#### **Intuition**

- The ACF for an Autoregressive process shows strong correlations until the lag `p` and begins to trail off afterwards.
- PACF only describes the relationship between an observation and its lag, suggesting that there may be no correlation for lag values beyond `k`

### **Moving Average Processes**

**Prose** - TO DO

In [ ]:
ar1 = np.array([1])
ma1 = np.array([1, -0.9])

MA_object1 = ArmaProcess(ar1, ma1)
simulated_data1 = MA_object1.generate_sample(nsample=1000)
plt.plot(simulated_data1)
plt.show();

In [ ]:
plot_acf(simulated_data1, lags=10)
plt.show();

In [ ]:
plot_pacf(simulated_data1, lags=10)
plt.show();

#### **Intuition**

- 

### **Forecasting with ARMA**

**Prose**

In [ ]:
# Loading data
xdat = pd.read_csv(data + 'savings_change.csv')

xdat.columns = ['date', 'value']
xdat['date'] = pd.to_datetime(xdat['date'])

xdat.set_index('date').plot();

In [ ]:
# Seasonal decompostion
decomp = sm.tsa.seasonal_decompose(xdat['value'], period=12, model='additive')
figure = decomp.plot()
plt.show();

In [ ]:
# Formal testing of stationarity with the ADF test
val = xdat['value']
result = adfuller(val)

print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'Critical Values: ')
for k, v in result[4].items():
    print(f'\t{k}: {v.round(4)}')

In [ ]:
plot_acf(val, lags=25); print()

In [ ]:
plot_pacf(val, lags=25); print()

In [ ]:
# Splitting the data into training and validation, reserving the last 3 for testing
xtrain, xvalid = model_selection.train_test_split(val, test_size=12)

`pmdarima` also contains a Fourier featurizer which:
- allows users to capture seasonality in the model without worrying about the _seasonal order_ (for pure ARIMA `seasonality=False`).
- Creates a set of covariates based on Fourier decomposition.
- allows the inclusion of arbitrary length seasonal patterns.
- can handle short-term dynamics with a straight-forward ARMA error.

In [ ]:
# Create pipeline, pre-process features and fit using the pmdarima lib
pipe = pipeline.Pipeline([
    ("fourier", ppc.FourierFeaturizer(m=4)),
    ("arima", arima.AutoARIMA(stepwise=True, trace=1, error_action="ignore",
                             seasonal=False, # using Fourier tfms
                             suppress_warnings=True))
])

pipe.fit(xtrain)

In [ ]:
pipe.summary()

In [ ]:
preds, conf_int = pipe.predict(n_periods=xvalid.shape[0], return_conf_int=True)
preds = list(preds)
print("Forecasts:\n", preds)

In [ ]:
print("Confidence Intervals:\n", conf_int)

In [ ]:
# Visualize the forecast
xvalid = pd.DataFrame(xvalid.values, columns=['actual'])
xvalid['predicted'] = preds
xvalid.plot();

This model's outputs can be useful directionally useful as a baseline, but not much else.

> there is a predictable lag: one period after an increase in the original series the forecast goes up as well; this is due to autoregressive nature of the model.

> the model captures the general dynamics of the model, but struggles with the range of values - this is a consequence of the constant variance assumption.


## **ARIMA - Modelling Beyond ARMA**

**PROSE - TODO**

In [ ]:
# Let's load Tesla's daily closing stock price on the NYSE
xdat = pd.read_csv(data + 'tesla_prices_5y.csv', usecols=['Date', 'Close'])
xdat.columns = ['date', 'value']

xdat['date'] = pd.to_datetime(xdat['date'])
xdat.set_index('date').plot();

In [ ]:
# Applying Dickey-Fuller test to check for stationarity
val = xdat['value']
result = adfuller(val)

print(f'ADF Statistic: {result[0].round(4)}')
print(f'p-value: {result[1].round(4)}')
print(f'Critical Values: ')
for k, v in result[4].items():
    print(f'\t{k}: {v.round(4)}')

In [ ]:
# Using the diff operator to take the difference 
ydat = xdat['value'].diff()
ydat.plot()
print()

In [ ]:
# Again, checking for stationarity
result = adfuller(ydat.dropna())

print(f'ADF Statistic: {result[0].round(4)}')
print(f'p-value: {result[1].round(4)}')
print(f'Critical Values: ')
for k, v in result[4].items():
    print(f'\t{k}: {v.round(4)}')

**PROSE - TO DO**

## **SARIMA** - _Incorporating Seasonality_

In [ ]:
series = pd.read_csv(data + 'passengers.csv')
series['date'] = pd.to_datetime(series['date'])

decomp = seasonal_decompose(series['passengers'], period=12, model='multiplicative') 
figure = decomp.plot()
plt.show();

In [ ]:
# Check stationarity
result = adfuller(decomp.seasonal)

print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'Critical Values: ')
for k, v in result[4].items():
    print(f'\t{k}: {v.round(4)}')

In [ ]:
# Plotting PACF
plot_pacf(decomp.seasonal, lags=12); print()

In [ ]:
# Plot ACF
plot_acf(decomp.seasonal, lags=12); print()

## **Full Pipeline -  Tackling a _Real World_ Problem**

In order to put all the moving parts together, let's build a complete pipeline which is based on the Demand Forecasting competition on Kaggle.

In [ ]:
#!kaggle competitions download -c demand-forecasting-kernels-only -p data/demand-forecasting 

In [ ]:
from pathlib import Path
import zipfile
cred_path = Path('~/.kaggle/kaggle.json').expanduser()

def download_data(dataset, path):
    # Create local folder if one doesn't exist
    os.makedirs(path, exist_ok=True)
    # Call Kaggle API and switch to competitions 
    kaggle_call = f"kaggle competitions download -c {dataset} -p {path}"
    subprocess.run(kaggle_call, shell=True, check=True)
    # Unzip since CLI wasn't working properly
    for file in Path(path).glob("*.zip"):
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(path)
        print(f"Extracted {file}")
        file.unlink()
        
    print(f"Dataset '{dataset}' has been downloaded to '{path}'")

# Download additional datasets from Konrad's NB
dataset = "demand-forecasting-kernels-only"
download_path = "data/demand-forecasting"

download_data(dataset, download_path)

In [ ]:
train = pd.read_csv("data/demand-forecasting/train.csv", parse_dates=['date'], index_col='date')
test = pd.read_csv("data/demand-forecasting/test.csv", parse_dates=['date'], index_col='date')
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
# Concat files
df = pd.concat([train, test], sort=True)

In [ ]:
# Subsetting to one item and store combo
buf = df[(df.item==1) & (df.store==1)].copy()
buf.head()

In [ ]:
# Assess components
decomposition = seasonal_decompose(buf.sales.dropna(), period=365)
figure = decomposition.plot()
plt.show()

Given that we have four years worth of data (so that gives us more than 2 complete cycles), we can model seasonal effects for this exercise.

In [ ]:
# Skipping the first 2 quarters to align with start of 2014
tr_start, tr_end = '2014-01-01', '2017-09-30'
te_start, te_end = '2017-10-01', '2017-12-31'

x0 = buf['sales'][tr_start:tr_end].dropna()
x1 = buf['sales'][te_start:te_end].dropna()

In [ ]:
x0.shape, x1.shape

In [ ]:
plot_acf(x0, lags=12); print()
plot_pacf(x0, lags=12); print()

Conducting a grid search for model order coefficients.

In [ ]:
model_autoARIMA = auto_arima(x0, start_p=7, start_q=7, test='adf', 
                             max_p=7, max_q=7,
                             m=7, d=1,
                             seasonal=True,
                             start_P=1, D=1,
                             trace=True,
                             error_action='ignore',
                             suppress_warnings=True,
                             stepwise=True)

In [ ]:
print(model_autoARIMA.summary())

In [ ]:
model_autoARIMA.plot_diagnostics()
plt.show();

In [ ]:
pred = model_autoARIMA.predict(x1.shape[0])

pd.DataFrame({'test': x1, 'pred': pred}).plot()
plt.show();

While the model does a decent job of capturing the the dynamics of this particular time series, it can't seem to adjust for variations in the ranges of values, especially for more recent data.

## **Comparative Performance with Prophet**

It is always useful to compare "classic" approaches with more recent models, like Prophet. While the latter has been dicredited for its inability to produce probabilistic forecasts, leading to some spurious predictions, it should nonetheless be an informative exercise.

In [ ]:
stock = 'TATASTEEL'

NOTE - Cleanup-- merge this function and the previous one 

In [ ]:
def download_data(dataset, path):
    # Create local folder if one doesn't exist
    os.makedirs(path, exist_ok=True)
    # Call Kaggle API
    kaggle_call = f"kaggle datasets download -d {dataset} -p {path} --unzip"
    subprocess.run(kaggle_call, shell=True, check=True)
    print(f"Dataset '{dataset}' has been downloaded to '{path}'")

# Download additional datasets from Konrad's NB
dataset = "rohanrao/nifty50-stock-market-data"
download_path = "data/nifty50-stocks"

download_data(dataset, download_path)

In [ ]:
df = pd.read_csv('data/nifty50-stocks/' + stock + '.csv')
df.set_index("Date", drop=False, inplace=True)

df.head(5)

In [ ]:
df.Symbol.value_counts()

In [ ]:
# Plotting the target
df.VWAP.plot();

In [ ]:
df.head()

In [ ]:
# Constructing basic features for the purpose of explanatory variables.
# Rolling statistics over the last 3 days for the price and volume dynamics
lag_features = ["High", "Low", "Volume"]
window_size = 3

df_rolled = df[lag_features].rolling(window=window_size, min_periods=0)
df_mean = df_rolled.mean().shift(1).reset_index()
df_std = df_rolled.std().shift(1).reset_index()

for feature in lag_features:
    df[feature + '_mean_lag' + str(window_size)] = df_mean[feature].values
    df[feature + '_std_lag' + str(window_size)] = df_std[feature].values

In [ ]:
df.isna().sum()

In [ ]:
means = ['Trades', 'Deliverable Volume', '%Deliverble', 'High_mean_lag3', 'High_std_lag3', 'Low_mean_lag3',
         'Low_std_lag3', 'Volume_mean_lag3', 'Volume_std_lag3']

In [ ]:
df[means].mean()

In [ ]:
# Replace Nans with means
col_means = df[means].mean()
df.fillna(col_means, inplace=True)

In [ ]:
# Verifying results
df.isna().sum()

In [ ]:
# Prepare training and validation splits
df_train = df[df.Date < "2019"]
df_valid = df[df.Date >= "2019"]

exogenous_features = ['High_mean_lag3', 'High_std_lag3', 'Low_mean_lag3',
       'Low_std_lag3', 'Volume_mean_lag3', 'Volume_std_lag3',]

In [ ]:
df_train.VWAP.describe()

In [ ]:
# Fitting Arima
model_arima = auto_arima(df_train.VWAP, exogenous=df_train[exogenous_features],
                         m=7,
                         # Ranges for p,q,P,Q parameters - flexible
                         max_p=2, max_q=2,
                         max_P=1, max_Q=1,
                         trace=True, error_action="ignore", suppress_warnings=True)
model_arima.fit(df_train.VWAP, exogenous=df_train[exogenous_features])

forecast = model_arima.predict(n_periods=len(df_valid), exogenous=df_valid[exogenous_features])

In [ ]:
forecast.describe()

In [ ]:
# Fit the data to a Prophet model using default parameters
df = df_train[['Date', 'VWAP']].rename(columns={"Date": "ds", "VWAP": "y"})

model_prophet = Prophet()

# Adding exogenous variables
for f in exogenous_features:
    df[f] = df_train[f]
    model_prophet.add_regressor(f)

model_prophet.fit(df)

In [ ]:
# Renaming cols for plotting
forecast = model_prophet.predict(df_valid[["Date", "VWAP"] + exogenous_features].rename(columns={"Date": "ds"}))
df_valid["Forecast_Prophet"] = forecast.yhat.values

In [ ]:
df_valid[["VWAP", "Forecast_ARIMAX", "Forecast_Prophet"]].plot();

In [ ]:
df_valid